In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os

from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
# Set up constants
IMG_SIZE = (224, 224)  # VGG16 input size
BATCH_SIZE = 32
EPOCHS = 100  # Increased max epochs, early stopping will prevent unnecessary training

# Define the path to the images folder
data_dir = '/content/drive/My Drive/images'
print("Contents of data_dir:")
print(os.listdir(data_dir))

# Update class names to include SCC
class_names = ['MEL', 'NV', 'BCC', 'SCC']
for class_name in class_names:
    if not os.path.isdir(os.path.join(data_dir, class_name)):
        raise ValueError(f"Folder {class_name} not found in {data_dir}")



Contents of data_dir:
['NV', 'MEL', 'BCC', 'SCC']


In [ ]:
# Set up data generators with increased augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    classes=class_names,
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    classes=class_names,
    shuffle=True
)

# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))



Found 7190 images belonging to 4 classes.
Found 1795 images belonging to 4 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Fine-tune the model
for layer in base_model.layers:
    layer.trainable = True

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(4, activation='softmax')(x)  # 4 classes now

# Create the final model
model = Model(inputs=base_model.input, outputs=output)



In [ ]:
# Define F1 Score metric as a class
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()



In [ ]:
# Compile the model with additional metrics
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall(), F1Score()])

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
class_weight_dict = dict(enumerate(class_weights))

# Define callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with class weights and callbacks
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    class_weight=class_weight_dict,
    callbacks=[reduce_lr, early_stopping]
)

# Save the model
model.save('/content/drive/My Drive/models/skin_lesion_classifier_vgg16_all_classes.h5')



Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


224/224 ━━━━━━━━━━━━━━━━━━━━ 2261s 10s/step - accuracy: 0.2788 - f1_score: 0.0047 - loss: 1.3824 - precision: 0.1822 - recall: 0.0024 - val_accuracy: 0.0703 - val_f1_score: 0.0000e+00 - val_loss: 1.4114 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 2/100
  1/224 ━━━━━━━━━━━━━━━━━━━━ 1:43 466ms/step - accuracy: 0.2500 - f1_score: 0.0000e+00 - loss: 1.8435 - precision: 0.0000e+00 - recall: 0.0000e+00

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


224/224 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.2500 - f1_score: 0.0000e+00 - loss: 1.8435 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_f1_score: 0.0000e+00 - val_loss: 1.4354 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 3/100
224/224 ━━━━━━━━━━━━━━━━━━━━ 253s 1s/step - accuracy: 0.3725 - f1_score: 0.0388 - loss: 1.2885 - precision: 0.7009 - recall: 0.0200 - val_accuracy: 0.0826 - val_f1_score: 0.0000e+00 - val_loss: 1.4094 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 4/100
224/224 ━━━━━━━━━━━━━━━━━━━━ 35s 156ms/step - accuracy: 0.0938 - f1_score: 0.0588 - loss: 1.5047 - precision: 0.5000 - recall: 0.0312 - val_accuracy: 0.0000e+00 - val_f1_score: 0.0000e+00 - val_loss: 1.3819 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 5/100
224/224 ━━━━━━━━━━━━━━━━━━━━ 272s 1s/step - accuracy: 0.3348 - f1_score: 0.0427 - loss: 1.2737 -

In [ ]:
# Print class indices
print("Class indices:", train_generator.class_indices)

# Function to predict image
def predict_image(img_path, nv_threshold=0.7):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)

    if np.argmax(prediction) == class_names.index('NV') and prediction[0][class_names.index('NV')] < nv_threshold:
        predicted_class = class_names[np.argsort(prediction[0])[-2]]
    else:
        predicted_class = class_names[np.argmax(prediction)]

    confidence = np.max(prediction)

    return predicted_class, confidence

# Directory containing test images
finish_dir = '/content/drive/My Drive/finish'



Class indices: {'MEL': 0, 'NV': 1, 'BCC': 2, 'SCC': 3}


In [ ]:
# Interactive prediction loop
while True:
    user_input = input("Enter an image number (1-1000) or 'q' to quit: ")

    if user_input.lower() == 'q':
        break

    try:
        image_number = int(user_input)

        for filename in os.listdir(finish_dir):
            if filename.startswith(f"{image_number}.") and filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(finish_dir, filename)

                predicted_class, confidence = predict_image(img_path)

                print(f"Image: {filename}")
                print(f"Predicted class: {predicted_class}")
                print(f"Confidence: {confidence:.2f}")
                print()
                break
        else:
            print(f"No image found with number {image_number}")

    except ValueError:
        print("Invalid input. Please enter a number or 'q' to quit.")

print("Thank you for using the classifier!")

Enter an image number (1-1000) or 'q' to quit: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Image: 1.jpg
Predicted class: BCC
Confidence: 0.69

Enter an image number (1-1000) or 'q' to quit: 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Image: 2.jpg
Predicted class: BCC
Confidence: 0.67

Enter an image number (1-1000) or 'q' to quit: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Image: 3.jpg
Predicted class: BCC
Confidence: 0.55

Enter an image number (1-1000) or 'q' to quit: 4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Image: 4.jpg
Predicted class: BCC
Confidence: 0.71

Enter an image number (1-1000) or 'q' to quit: 5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Image: 5.jpg
Predicted class: BCC
Confidence: 0.66

Enter an image number (1-1000) or 'q' to quit: 6
No image found with number 6
Enter an image number (1-1000) or 'q' to quit: 6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Image: 6.jpg
Predicted class: BCC
Confidence: 0.66

Enter an image number (1-1000) or 'q' to quit: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Ima

KeyboardInterrupt: Interrupted by user

EVALUATION


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Define the F1Score metric (same as in your training code)
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

# Load the saved model
model = load_model('/content/drive/My Drive/models/skin_lesion_classifier_vgg16_all_classes.h5',
                   custom_objects={'F1Score': F1Score})

# Set up constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Define the path to the images folder
data_dir = '/content/drive/My Drive/images'

# Set up data generator for evaluation
eval_datagen = ImageDataGenerator(rescale=1./255)

eval_generator = eval_datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Evaluate the model
scores = model.evaluate(eval_generator, verbose=1)

# Print the results
print("Evaluation on the entire dataset:")
for metric, score in zip(model.metrics_names, scores):
    print(f"{metric}: {score}")

# If you want to evaluate on specific classes
class_names = ['MEL', 'NV', 'BCC', 'SCC']
for i, class_name in enumerate(class_names):
    class_generator = eval_datagen.flow_from_directory(
        data_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        classes=[class_name],
        shuffle=False
    )
    scores = model.evaluate(class_generator, verbose=0)
    print(f"\nEvaluation on {class_name} class:")
    for metric, score in zip(model.metrics_names, scores):
        print(f"{metric}: {score}")

Found 8985 images belonging to 4 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


281/281 ━━━━━━━━━━━━━━━━━━━━ 2561s 9s/step - accuracy: 0.0680 - f1_score: 0.0437 - loss: 3.1579 - precision: 0.0470 - recall: 0.0409
Evaluation on the entire dataset:
loss: 3.615205764770508
compile_metrics: 0.11285475641489029
Found 1113 images belonging to 1 classes.


InvalidArgumentError: Graph execution error:

Detected at node LogicalAnd defined at (most recent call last):
<stack traces unavailable>
Incompatible shapes: [1,32] vs. [1,128]
	 [[{{node LogicalAnd}}]]
	tf2xla conversion failed while converting __inference_one_step_on_data_12831[]. Run with TF_DUMP_GRAPH_PREFIX=/path/to/dump/dir and --vmodule=xla_compiler=2 to obtain a dump of the compiled functions.
	 [[StatefulPartitionedCall]] [Op:__inference_one_step_on_iterator_12930]